In [50]:
from langchain import hub

# OpenAI Chat Model
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

# Create Retriever as tool for agent
from langchain.tools.retriever import create_retriever_tool

# Create TavilySearch as Tool for Agent
from langchain.tools.tavily_search import TavilySearchResults

# Create agent
from langchain.agents import create_openai_functions_agent

# Create Agent Executor
from langchain.agents import AgentExecutor

# Others
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, AIMessage

In [34]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

In [35]:
splitter = RecursiveCharacterTextSplitter()
documents = splitter.split_documents(docs)

In [36]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [37]:
vector_store = FAISS.from_documents(documents=documents,embedding=embeddings)

In [38]:
retriever = vector_store.as_retriever()

In [39]:
# Create Retriever as Tool

retriever_tool = create_retriever_tool(
                                        retriever=retriever,
                                        name="langsmith_search",
                                        description="Search for information about langSmith."
                                    )


In [40]:
# Create TavilySearch Tool

tavily_tool = TavilySearchResults()

In [41]:
tools = [retriever_tool, tavily_tool]

In [42]:
prompt = hub.pull("hwchase17/openai-functions-agent")
print(prompt)

input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='

In [44]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125")

#Create agent
agent = create_openai_functions_agent(llm=chat_model,tools=tools,prompt=prompt)

#Create agent executor
agentExecutor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [45]:
agentExecutor.invoke({"input":"how can langsmith help with testing?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can Langsmith help with testing'}`


Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping​Prototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.
The ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phas

{'input': 'how can langsmith help with testing?',
 'output': 'LangSmith is a platform for LLM application development, monitoring, and testing. Here are some ways LangSmith can help with testing:\n\n1. Prototyping: LangSmith supports quick experimentation between prompts, model types, retrieval strategy, and other parameters for LLM applications.\n   \n2. Debugging: LangSmith provides clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues when things go wrong.\n\n3. Initial Test Set: Developers can create datasets of inputs and reference outputs to run tests on LLM applications, making it easy to run custom evaluations and score test results.\n\n4. Comparison View: LangSmith offers a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of an application.\n\n5. Playground: LangSmith provides a playground environment for rapid iteration and experimenta

In [47]:
agentExecutor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1713401856, 'localtime': '2024-04-17 17:57'}, 'current': {'last_updated_epoch': 1713401100, 'last_updated': '2024-04-17 17:45', 'temp_c': 19.4, 'temp_f': 66.9, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 18.6, 'wind_kph': 29.9, 'wind_degree': 290, 'wind_dir': 'WNW', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 57, 'cloud': 75, 'feelslike_c': 19.4, 'feelslike_f': 66.9, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 5.0, 'gust_mph': 23.0, 'gust_kph': 37.1}}"}, {'url': 'https://www.weathertab.com/en/c/

{'input': 'what is the weather in SF?',
 'output': 'The current weather in San Francisco is partly cloudy with a temperature of 66.9°F (19.4°C). The wind speed is 29.9 kph coming from the west-northwest direction. The humidity is at 57%, and there is no precipitation at the moment.'}

In [51]:
#Conversation History
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agentExecutor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...
LangSmith can help test your LLM applications by providing you with language support and assistance during the testing process. LangSmith can help you identify language-related issues, provide feedback on language usage, and ensure that your applications are clear, concise, and effectively communicate your message. LangSmith can also help you improve the overall quality of your applications by offering suggestions for improvements and helping you refine your language skills.

> Finished chain.


{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith can help test your LLM applications by providing you with language support and assistance during the testing process. LangSmith can help you identify language-related issues, provide feedback on language usage, and ensure that your applications are clear, concise, and effectively communicate your message. LangSmith can also help you improve the overall quality of your applications by offering suggestions for improvements and helping you refine your language skills.'}